In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# FraudFinder - Feature Engineering (batch)

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?&download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/02_feature_engineering_batch.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[FraudFinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the FraudFinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

As you engineer features for model training, it's important to consider how the features are computed when making predictions with new data. For online predictions, you may have features that can be pre-computed via _batch feature engineering_. You may also features that need to be computed on-the-fly via _streaming-based feature engineering_. For these Fraudfinder labs, for computing features based on the last n _days_, you will use _batch_ feature engineering in BigQuery; for computing features based on the last n _minutes_, you will use _streaming-based_ feature engineering using Dataflow.

This notebook shows how to generate new features on bank transactions by customer and terminal over the last n days, by doing batch feature engineering in SQL with BigQuery. Then, you will create a feature store using Vertex AI Feature Store, and ingest your newly-created features from BigQuery into Vertex AI Feature Store, so that a feature store can become the single source of data for both training and model inference. 


This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)


Steps performed in this notebook:

- Build customer and terminal-related features
- Create Feature store, entities and features
- Ingest feature values in Feature store from BigQuery table
- Read features from the feature store

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [79]:
try:
  with open("id_file.txt", "r") as f:
    ID = f.read().strip()
  print(f"Using ID '{ID}' from id_file.txt")
except FileNotFoundError:
  print("id_file.txt not found. Please make sure the file exists.")
  ID = None 

Using ID '67jy6' from id_file.txt


In [80]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env_{ID}.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "fraud123-438914-fraudfinder"
STAGING_BUCKET       = "fraud123-438914-staging-67jy6"
PROJECT              = "fraud123-438914"
REGION               = "us-central1"
ID                   = "67jy6"
FEATURESTORE_ID      = "fraudfinder_67jy6"
MODEL_NAME           = "ff_model"
ENDPOINT_NAME        = "ff_model_endpoint"
TRAINING_DS_SIZE     = "1000"
DATA_DIR             = "data"
TRAIN_DATA_DIR       = "train"
CUSTOMER_ENTITY      = "customer"
TERMINAL_ENTITY      = "terminal"
TARGET               = "tx_fraud"



### Import libraries

In [ ]:
# General
import datetime as dt
import json
import os
import random
import sys
import time
from datetime import datetime, timedelta
from typing import List, Union

# Data Engineering
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

# Vertex AI and Vertex AI Feature Store
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery
from google.cloud.aiplatform import EntityType, Feature, Featurestore

### Define constants

In [81]:
# Define the date range of transactions for feature engineering (last 10 days up until yesterday)
YESTERDAY = datetime.today() - timedelta(days=1)
YEAR_MONTH_PREFIX = YESTERDAY.strftime("%Y-%m")
DATAPROCESSING_START_DATE = (YESTERDAY - timedelta(days=10)).strftime("%Y-%m-%d")
DATAPROCESSING_END_DATE = YESTERDAY.strftime("%Y-%m-%d")

# Define BiqQuery dataset and tables to calculate features.
RAW_BQ_TRANSACTION_TABLE_URI = f"{PROJECT_ID}.tx.tx"
RAW_BQ_LABELS_TABLE_URI = f"{PROJECT_ID}.tx.txlabels"
FEATURES_BQ_TABLE_URI = f"{PROJECT_ID}.tx.wide_features_table"

# Define Vertex AI Feature store settings.
CUSTOMERS_TABLE_NAME = f"customers_{DATAPROCESSING_END_DATE.replace('-', '')}"
CUSTOMERS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{CUSTOMERS_TABLE_NAME}"
TERMINALS_TABLE_NAME = f"terminals_{DATAPROCESSING_END_DATE.replace('-', '')}"                    
TERMINALS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{TERMINALS_TABLE_NAME}"
ONLINE_STORAGE_NODES = 1
FEATURE_TIME = "feature_ts"
CUSTOMER_ENTITY_ID = "customer"
TERMINAL_ENTITY_ID = "terminal"

### Helpers

Define a set of helper functions to run BigQuery query and create features. 

In [82]:
def run_bq_query(sql: str, show=False) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
        show: A flag to show query result in a Pandas Dataframe
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    result = client_result.result()
    print(f"Finished job_id: {job_id}")
    
    if show:
        df = result.to_arrow().to_pandas()
        return df

## Feature Engineering

### Define customer and terminal-related features for batch feature engineering

In this section, you will create features, based on historical customer behaviour and historical terminal activities. This features will be batch-generated using SQL in BigQuery, where the historical data is stored.

The query below will calculate 2 sets of features: 

1. **Customer-related features**: which describes the spending behaviour of customer within 1, 7 and 15 days time windows using number of transactions and average amount spent in dollars ($)

2. **Terminal-related features** which describes the risk of a given terminal to be exposed to fraudulent transactions within 1, 7 and 15 days using average number of fraudulent transactions in dollars ($), the number of transactions and risk index. One thing to note is that you will add some delay to take into account time that would pass between the time of transaction and the result of fraud investigation or customer claim.

You will use one month of transaction data starting from the end of January and going back to compute the features.

Below is the schema you should expect to see, after doing the batch feature engineering in BigQuery:

|feature_time           |customer_id| customer batch features   |
|-----------------------|-----------|---------------------------|
|2022-01-01 17:20:15 UTC|1          |(e.g., nb_tx,  avg_tx)     |
|2022-01-02 12:08:40 UTC|2          |(e.g., nb_tx,  avg_tx)     |
|2022-01-03 17:30:48 UTC|3          |(e.g., nb_tx,  avg_tx)     |


|feature_time           |terminal_id| terminal batch features|
|-----------------------|-----------|------------------------|
|2022-01-01 17:20:15 UTC|12345      |(e.g., risk_x_days)     |
|2022-01-02 12:08:40 UTC|26789      |(e.g., risk_x_days)     |
|2022-01-03 17:30:48 UTC|101112     |(e.g., risk_x_days)     |


#### Create the query to create batch features

Date settings to be used:

In [83]:
print(f"""
DATAPROCESSING_START_DATE: {DATAPROCESSING_START_DATE}
DATAPROCESSING_END_DATE: {DATAPROCESSING_END_DATE}
""")


DATAPROCESSING_START_DATE: 2024-11-06
DATAPROCESSING_END_DATE: 2024-11-16



##### Customer feature table

Customer table SQL query string:

In [84]:
create_customer_batch_features_query = f"""
CREATE OR REPLACE TABLE `{CUSTOMERS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  -- query to calculate CUSTOMER spending behaviour --------------------------------------------------------------------------------
  get_customer_spending_behaviour AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TX_AMOUNT,
    TX_FRAUD,
    
    # calc the number of customer tx over daily windows per customer (1, 7 and 15 days, expressed in seconds)
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_1DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_7DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_14DAY_WINDOW,
      
    # calc the customer average tx amount over daily windows per customer (1, 7 and 15 days, expressed in seconds, in dollars ($))
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW,
  FROM get_raw_table)

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  CUSTOMER_ID AS customer_id,
  CAST(CUSTOMER_ID_NB_TX_1DAY_WINDOW AS INT64) AS customer_id_nb_tx_1day_window,
  CAST(CUSTOMER_ID_NB_TX_7DAY_WINDOW AS INT64) AS customer_id_nb_tx_7day_window,
  CAST(CUSTOMER_ID_NB_TX_14DAY_WINDOW AS INT64) AS customer_id_nb_tx_14day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_1day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_7day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_14day_window,
FROM
  get_customer_spending_behaviour
"""

##### Run the query 

You create the customer features table

In [85]:
run_bq_query(create_customer_batch_features_query)

Finished job_id: 6e03dc4a-2352-41cc-90f2-5c7becb130b1


##### Inspect the result 

You can query some data rows to validate the result of the query

In [86]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: 501901c3-758f-412d-8141-c3255ca289f1


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window
0,2024-11-08 11:03:35+00:00,1791201696340742,1,3,3,16.250000,17.763333,17.763333
1,2024-11-02 18:44:30+00:00,2307821961924105,2,3,3,19.290000,21.296667,21.296667
2,2024-11-07 22:28:52+00:00,2313731463981676,2,10,10,48.870000,53.277000,53.277000
3,2024-11-15 22:01:57+00:00,6231179606200312,10,33,60,91.073000,94.958485,94.732833
4,2024-11-08 04:21:18+00:00,8450121756479199,5,20,21,14.800000,12.033500,11.665238
5,2024-11-02 04:50:13+00:00,9699840033744621,4,5,5,50.975000,50.844000,50.844000
6,2024-11-09 04:59:36+00:00,0691045050891459,9,30,33,34.437778,33.209667,33.209697
7,2024-11-13 08:03:44+00:00,2699086214229169,3,9,18,63.780000,62.731111,61.732222
8,2024-11-16 03:49:50+00:00,3037379089896585,3,9,19,10.500000,9.674444,9.287368
9,2024-11-09 01:09:19+00:00,3337712790653467,4,31,36,55.857500,55.629677,54.914722


##### Terminal feature table

Terminal table SQL query string:

In [87]:
create_terminal_batch_features_query = f"""
CREATE OR REPLACE TABLE `{TERMINALS_BQ_TABLE_URI}` AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  # query to calculate TERMINAL spending behaviour --------------------------------------------------------------------------------
  get_variables_delay_window AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    
    # calc total amount of fraudulent tx and the total number of tx over the delay period per terminal (7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_DELAY,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_TX_DELAY,
      
    # calc total amount of fraudulent tx and the total number of tx over the delayed window per terminal (window + 7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_1_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_7_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_14_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_TX_1_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_TX_7_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_TX_14_DELAY_WINDOW,
  FROM get_raw_table),

  # query to calculate TERMINAL risk factors ---------------------------------------------------------------------------------------
  get_risk_factors AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    # calculate numerator of risk index
    NB_FRAUD_1_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_1DAY_WINDOW,
    NB_FRAUD_7_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_7DAY_WINDOW,
    NB_FRAUD_14_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_14DAY_WINDOW,
    # calculate denominator of risk index
    NB_TX_1_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_1DAY_WINDOW,
    NB_TX_7_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_7DAY_WINDOW,
    NB_TX_14_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_14DAY_WINDOW,
      FROM
    get_variables_delay_window),

  # query to calculate the TERMINAL risk index -------------------------------------------------------------------------------------
  get_risk_index AS (
    SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TERMINAL_ID_NB_TX_1DAY_WINDOW,
    TERMINAL_ID_NB_TX_7DAY_WINDOW,
    TERMINAL_ID_NB_TX_14DAY_WINDOW,
    # calculate the risk index
    (TERMINAL_ID_NB_FRAUD_1DAY_WINDOW/(TERMINAL_ID_NB_TX_1DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_1DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_7DAY_WINDOW/(TERMINAL_ID_NB_TX_7DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_7DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_14DAY_WINDOW/(TERMINAL_ID_NB_TX_14DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_14DAY_WINDOW
    FROM get_risk_factors 
  )

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  TERMINAL_ID AS terminal_id,
  CAST(TERMINAL_ID_NB_TX_1DAY_WINDOW AS INT64) AS terminal_id_nb_tx_1day_window,
  CAST(TERMINAL_ID_NB_TX_7DAY_WINDOW AS INT64) AS terminal_id_nb_tx_7day_window,
  CAST(TERMINAL_ID_NB_TX_14DAY_WINDOW AS INT64) AS terminal_id_nb_tx_14day_window,
  CAST(TERMINAL_ID_RISK_1DAY_WINDOW AS FLOAT64) AS terminal_id_risk_1day_window,
  CAST(TERMINAL_ID_RISK_7DAY_WINDOW AS FLOAT64) AS terminal_id_risk_7day_window,
  CAST(TERMINAL_ID_RISK_14DAY_WINDOW AS FLOAT64) AS terminal_id_risk_14day_window,
FROM
  get_risk_index
"""

##### Run the query 

You create the customer features table

In [88]:
run_bq_query(create_terminal_batch_features_query)

Finished job_id: 963be79f-2ea5-49a1-b480-b99c72c45bee


##### Inspect the result 

You can query some data rows to validate the result of the query

In [89]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: 137830fc-1fa1-49c6-8a86-3f5f9ed215af


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window
0,2024-11-01 00:57:43+00:00,11564382,0,0,0,0.000000,0.000000,0.000000
1,2024-11-15 11:40:43+00:00,11564382,24,132,143,0.000000,0.000000,0.000000
2,2024-11-15 22:49:43+00:00,38374436,15,167,187,0.000000,0.023952,0.021390
3,2024-11-12 19:01:25+00:00,42532467,15,75,75,0.000000,0.013333,0.013333
4,2024-11-12 18:54:49+00:00,58340576,20,93,93,0.000000,0.000000,0.000000
5,2024-11-11 23:07:43+00:00,77719928,25,101,101,0.000000,0.000000,0.000000
6,2024-11-10 07:19:54+00:00,36523735,17,42,42,0.000000,0.119047,0.119047
7,2024-11-03 23:46:59+00:00,42396973,0,0,0,0.000000,0.000000,0.000000
8,2024-11-12 05:41:51+00:00,86294151,22,71,71,0.000000,0.014084,0.014084
9,2024-11-11 12:22:59+00:00,16220166,15,62,62,0.066666,0.032258,0.032258


### Define customer and terminal-related features for _real-time_ feature engineering

To make more accurate predictions, you can also create real-time features to inspect, for example, the most recent minutes of activity for both customers and terminals.

In order to have these features available for training and inference, we first need to make sure they are defined in a BigQuery table as placeholders to be ingested into Vertex AI Feature Store.

In the query below, you initialize two sets of features for real-time feature engineering: 

1. Customer features which describes the spending behaviour of customer within 15, 30 and 60 minutes time windows using number of transactions and average amount spent in dollars ($)

2. Terminal features which describes the risk of a given terminal to be exposed to fraudulent transactions within 15, 30 and 60 minutes using average number of fraudulent transactions in dollars ($) and the number of transactions. 

To do so, you will:

- Add one column for each real time feature
- Set 0 as default values for each of them
- Update all real-time columns with default values

Then you will create the actual values for real-time feature engineering in the next `03_feature_engineering_streaming.ipynb` notebook. 

#### Define the query to initialize the real-time features.

##### Customer feature table

Customer table SQL query string:

In [90]:
initiate_real_time_customer_features_query = f"""
ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ADD COLUMN customer_id_nb_tx_15min_window INT64,
ADD COLUMN customer_id_nb_tx_30min_window INT64,
ADD COLUMN customer_id_nb_tx_60min_window INT64,
ADD COLUMN customer_id_avg_amount_15min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_30min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ALTER COLUMN customer_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{CUSTOMERS_BQ_TABLE_URI}`
SET customer_id_nb_tx_15min_window = 0,
    customer_id_nb_tx_30min_window  = 0,
    customer_id_nb_tx_60min_window  = 0, 
    customer_id_avg_amount_15min_window = 0,
    customer_id_avg_amount_30min_window  = 0,
    customer_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

In [91]:
initiate_real_time_terminal_features_query = f"""
ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ADD COLUMN terminal_id_nb_tx_15min_window INT64,
ADD COLUMN terminal_id_nb_tx_30min_window INT64,
ADD COLUMN terminal_id_nb_tx_60min_window INT64,
ADD COLUMN terminal_id_avg_amount_15min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_30min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ALTER COLUMN terminal_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{TERMINALS_BQ_TABLE_URI}`
SET terminal_id_nb_tx_15min_window = 0,
    terminal_id_nb_tx_30min_window  = 0,
    terminal_id_nb_tx_60min_window  = 0,
    terminal_id_avg_amount_15min_window = 0,
    terminal_id_avg_amount_30min_window = 0,
    terminal_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

#### Run the query above to initialize the real-time features.

In [92]:
for query in [initiate_real_time_customer_features_query, initiate_real_time_terminal_features_query]:
    run_bq_query(query)

Finished job_id: 94b6de5a-c809-4e97-857f-0878e8ab230f
Finished job_id: 158f6cbc-1f87-4bbf-883b-2c52f9178a60


#### Inspect BigQuery features tables

In [93]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 9a1f7337-c129-413a-9da1-164459c0ec5e


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window,customer_id_nb_tx_15min_window,customer_id_nb_tx_30min_window,customer_id_nb_tx_60min_window,customer_id_avg_amount_15min_window,customer_id_avg_amount_30min_window,customer_id_avg_amount_60min_window
0,2024-11-02 08:32:15+00:00,4632248894433628,1,1,1,17.41,17.41,17.41,0,0,0,0.0,0.0,0.0
1,2024-11-01 01:29:49+00:00,8765219799666167,1,1,1,64.95,64.95,64.95,0,0,0,0.0,0.0,0.0
2,2024-11-01 05:04:39+00:00,2327921523631584,1,1,1,88.94,88.94,88.94,0,0,0,0.0,0.0,0.0
3,2024-11-01 13:52:34+00:00,9996676150729590,1,1,1,31.26,31.26,31.26,0,0,0,0.0,0.0,0.0
4,2024-11-02 06:54:31+00:00,8856675185185793,1,1,1,46.85,46.85,46.85,0,0,0,0.0,0.0,0.0


In [94]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 4658c534-0041-4675-aaea-9d3f094bde5b


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window,terminal_id_nb_tx_15min_window,terminal_id_nb_tx_30min_window,terminal_id_nb_tx_60min_window,terminal_id_avg_amount_15min_window,terminal_id_avg_amount_30min_window,terminal_id_avg_amount_60min_window
0,2024-11-05 20:35:46+00:00,52333526,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
1,2024-11-03 02:57:11+00:00,90488489,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
2,2024-11-04 02:16:46+00:00,84468515,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
3,2024-11-05 23:41:01+00:00,34608094,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
4,2024-11-07 14:06:14+00:00,66078702,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0


Let's look at the final schema of the features table:

In [95]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{CUSTOMERS_TABLE_NAME}'
""", show=True)

Finished job_id: 7197500f-54a8-4114-be30-6e95804d2fe6


,column_name,data_type
0,feature_ts,TIMESTAMP
1,customer_id,STRING
2,customer_id_nb_tx_1day_window,INT64
3,customer_id_nb_tx_7day_window,INT64
4,customer_id_nb_tx_14day_window,INT64
5,customer_id_avg_amount_1day_window,FLOAT64
6,customer_id_avg_amount_7day_window,FLOAT64
7,customer_id_avg_amount_14day_window,FLOAT64
8,customer_id_nb_tx_15min_window,INT64
9,customer_id_nb_tx_30min_window,INT64


In [96]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{TERMINALS_TABLE_NAME}'
""", show=True)

Finished job_id: db10dae9-6081-4a1b-be79-10eeab83e3ea


,column_name,data_type
0,feature_ts,TIMESTAMP
1,terminal_id,STRING
2,terminal_id_nb_tx_1day_window,INT64
3,terminal_id_nb_tx_7day_window,INT64
4,terminal_id_nb_tx_14day_window,INT64
5,terminal_id_risk_1day_window,FLOAT64
6,terminal_id_risk_7day_window,FLOAT64
7,terminal_id_risk_14day_window,FLOAT64
8,terminal_id_nb_tx_15min_window,INT64
9,terminal_id_nb_tx_30min_window,INT64


## Feature store for feature management

### What is a feature store?

The features generated are great examples of features that we can store the [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore). This is because:

- The features are needed for real-time prediction
- feature values in a feature store can be used for both training and serving
- if needed, features can be shared with other use cases beyond fraud detection

Vertex AI Feature Store provides a centralized repository for organizing, storing, and serving ML features. Using a central featurestore enables an organization to efficiently share, discover, and re-use ML features at scale, which can increase the velocity of developing and deploying new ML applications.

### Why would you like to set up it?

So far you've built and stored features in BigQuery. 

Now, in order to predict fraud, you want to serve those features in real-time with millisecond scale latency. In particular, when the ML gateway receives a prediction request for a specific transaction (including customer, terminal, and transaction ids), the system needs to fetch the features related to that transaction and pass them as inputs to the model for online prediction. As you can imagine, an analytical data warehouse such as BigQuery is not able to provide low-latency near real-time read operations. 

Vertex AI Feature Store provides a managed service for low latency scalable feature serving. It also provides a centralized feature repository with easy APIs to search and discover features, as well as feature monitoring capabilities to track drift and other quality issues. 

Vertex AI Feature Store uses a time series data model to store a series of values for features, which enables Vertex AI Feature Store to maintain feature values as they change over time and to support point-in-time queries of feature values. Feature Store organizes resources hierarchically (`Featurestore -> EntityType -> Feature`) in the following order: 

- **Featurestore**: the resource to contains entities and features.
    - **EntityType**: under a Featurestore, an EntityType describes an minimal data entry.
        - **Feature**: under an EntityType, a feature is an attribute of the EntityType. 


You must create these resources before you can ingest data into a Feature Store. 

In the follow section, you will use create a feature store using Vertex AI Feature Store, and ingest data into it to be used later for training and model inference.

### Initialize Vertex AI SDK

Initialize the Vertex AI SDK to get access to Vertex AI services programmatically. 

In [110]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

### Create featurestore, `fraudfinder_<ID>`

A featurestore is the top-level container for entity types, features, and feature values. Typically, an organization creates one shared featurestore for feature ingestion, serving, and sharing across all teams in the organization.

Below you create a `featurestore` resources with different labels. 

<div class="alert alert-block alert-info">
<b>NOTE: This proecess can take about 5 to 7 minutes</b>
</div>


In [107]:
from google.cloud import aiplatform

# Get your project ID from the environment
project_id = !gcloud config get-value project 2> /dev/null
project_id = project_id[0]

# Initialize the aiplatform library (replace with your project ID and location)
aiplatform.init(project=project_id, location="us-central1") 


print(f"Creating new feature store {FEATURESTORE_ID}.")
ff_feature_store = aiplatform.Featurestore.create(
        featurestore_id=f"{FEATURESTORE_ID}",
        online_store_fixed_node_count=ONLINE_STORAGE_NODES,
        labels={"team": "cymbal_bank", "app": "fraudfinder"},
        sync=True,
    )

Creating new feature store fraudfinder_67jy6.


AlreadyExists: 409 Featurestore `fraudfinder_67jy6` already exists in projects/fraud123-438914/locations/us-central1.

### Create the main entity types and their features

An entity type is a collection of semantically related features. You define your own entity types, based on the concepts that are relevant to your use case. 

In this case, you create `customer` and `transaction` entity types. 

#### Create the ```customer``` entity type 

In [99]:
try:
    # get entity type, if it already exists
    customer_entity_type = ff_feature_store.get_entity_type(entity_type_id=CUSTOMER_ENTITY_ID)
except:
    # else, create entity type
    customer_entity_type = ff_feature_store.create_entity_type(
        entity_type_id=CUSTOMER_ENTITY_ID, description="Customer Entity", sync=True
    )

#### Create features of the ```customer``` entity type

In [100]:
customer_feature_configs = {
    "customer_id_nb_tx_1day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last day",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_7day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 7 days",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_14day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 14 days",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_1day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last day",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_7day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 7 days",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_14day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 14 days",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_15min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_30min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_60min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 60 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_15min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_30min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_60min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 60 minutes",
        "labels": {"status": "passed"},
    },
}

In [101]:
customer_feature_ids = customer_entity_type.batch_create_features(
    feature_configs=customer_feature_configs, sync=True
)

Batch creating features EntityType entityType: projects/520607199607/locations/us-central1/featurestores/fraudfinder_67jy6/entityTypes/customer


FailedPrecondition: 400 Cannot create batch of features [customer_id_nb_tx_1day_window, customer_id_nb_tx_7day_window, customer_id_nb_tx_14day_window, customer_id_avg_amount_1day_window, customer_id_avg_amount_7day_window, customer_id_avg_amount_14day_window, customer_id_nb_tx_15min_window, customer_id_nb_tx_30min_window, customer_id_nb_tx_60min_window, customer_id_avg_amount_15min_window, customer_id_avg_amount_30min_window, customer_id_avg_amount_60min_window] under EntityType projects/520607199607/locations/us-central1/featurestores/fraudfinder_67jy6/entityTypes/customer, features [customer_id_avg_amount_14day_window, customer_id_avg_amount_15min_window, customer_id_nb_tx_7day_window, customer_id_nb_tx_30min_window, customer_id_avg_amount_60min_window, customer_id_avg_amount_1day_window, customer_id_nb_tx_15min_window, customer_id_avg_amount_30min_window, customer_id_nb_tx_14day_window, customer_id_nb_tx_60min_window, customer_id_nb_tx_1day_window, customer_id_avg_amount_7day_window

#### Create the ```terminal``` entity type

In [102]:
try:
    # get entity type, if it already exists
    terminal_entity_type = ff_feature_store.get_entity_type(entity_type_id=TERMINAL_ENTITY_ID)
except:
    # else, create entity type
    terminal_entity_type = ff_feature_store.create_entity_type(
    entity_type_id=TERMINAL_ENTITY_ID, description="Terminal Entity", sync=True
)

#### Create features of the ```terminal``` entity type

In [103]:
terminal_feature_configs = {
    "terminal_id_nb_tx_1day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last day",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_7day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the 7 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_14day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the 14 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_1day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last day",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_7day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last 7 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_14day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last 14 day",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_15min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_30min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_60min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 60 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_15min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_30min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_60min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 60 minutes",
        "labels": {"status": "passed"},
    },
}

In [104]:
terminal_feature_ids = terminal_entity_type.batch_create_features(
    feature_configs=terminal_feature_configs, sync=True
)

Batch creating features EntityType entityType: projects/520607199607/locations/us-central1/featurestores/fraudfinder_67jy6/entityTypes/terminal


FailedPrecondition: 400 Cannot create batch of features [terminal_id_nb_tx_1day_window, terminal_id_nb_tx_7day_window, terminal_id_nb_tx_14day_window, terminal_id_risk_1day_window, terminal_id_risk_7day_window, terminal_id_risk_14day_window, terminal_id_nb_tx_15min_window, terminal_id_nb_tx_30min_window, terminal_id_nb_tx_60min_window, terminal_id_avg_amount_15min_window, terminal_id_avg_amount_30min_window, terminal_id_avg_amount_60min_window] under EntityType projects/520607199607/locations/us-central1/featurestores/fraudfinder_67jy6/entityTypes/terminal, features [terminal_id_risk_14day_window, terminal_id_avg_amount_15min_window, terminal_id_nb_tx_7day_window, terminal_id_nb_tx_30min_window, terminal_id_avg_amount_60min_window, terminal_id_risk_1day_window, terminal_id_nb_tx_15min_window, terminal_id_avg_amount_30min_window, terminal_id_nb_tx_14day_window, terminal_id_nb_tx_60min_window, terminal_id_nb_tx_1day_window, terminal_id_risk_7day_window] already exist.

### Inspect your feature store in the Vertex AI console

You can also inspect your feature store in the [Vertex AI Feature Store console](https://console.cloud.google.com/vertex-ai/features)

### Ingest feature values in Vertex AI Feature Store

Now we need to ingest the actual feature values you created in BigQuery into the Vertex AI Feature Store.

To ingest features values in Vertex AI Feature Store, you need to check the following requirements related to **Source Data format and Layout**:

- Features values have to [be stored](https://cloud.google.com/vertex-ai/docs/featurestore/source-data) in BigQuery tables or Avro and CSV files on Google Cloud Storage.
- Each imported feature entity *must* have an ID.
- Each feature entity can *optionally* have a timestamp, to specifying when the feature values are generated.

In the following queries, you ingest feature values from those BigQuery tables into Vertex AI Feature Store.

#### Ingest customer feature values into `customers` entity in Vertex AI Feature Store 

In the following section, you will import customer feature values into your feature store.

In [111]:
CUSTOMERS_FEATURES_IDS = [
    feature.name for feature in customer_feature_ids.list_features()
]
CUSTOMER_BQ_SOURCE_URI = f"bq://{CUSTOMERS_BQ_TABLE_URI}"
CUSTOMER_ENTITY_ID_FIELD = "customer_id"

<div class="alert alert-block alert-info">
<b>NOTE: Creating Customer entity can take about 5 to 7 minutes</b>
</div>


In [112]:
customer_entity_type.ingest_from_bq(
    feature_ids=CUSTOMERS_FEATURES_IDS,
    feature_time=FEATURE_TIME,
    bq_source_uri=CUSTOMER_BQ_SOURCE_URI,
    entity_id_field=CUSTOMER_ENTITY_ID_FIELD,
    disable_online_serving=False,
    worker_count=10,
    sync=True,
)

Importing EntityType feature values: projects/520607199607/locations/us-central1/featurestores/fraudfinder_67jy6/entityTypes/customer
Import EntityType feature values backing LRO: projects/520607199607/locations/us-central1/featurestores/fraudfinder_67jy6/entityTypes/customer/operations/6455048495396028416


FailedPrecondition: 400 IAM settings are still propagating, please retry the request in a few mins or reach out to support upon continued failure. 9: IAM settings are still propagating, please retry the request in a few mins or reach out to support upon continued failure.

#### Monitor the `customer` features ingestion job in the console.


You can go to the [Feature Store Console](https://console.cloud.google.com/vertex-ai/ingestion-jobs) to view your ingestion job. 

#### Ingest terminal feature values into `terminal` entity in Vertex AI Feature Store 

In the following section, you will import terminal feature values into your feature store.

In [ ]:
TERMINAL_ENTITY_ID = "terminal"
TERMINALS_FEATURES_IDS = [
    feature.name for feature in terminal_feature_ids.list_features()
]
TERMINALS_BQ_SOURCE_URI = f"bq://{TERMINALS_BQ_TABLE_URI}"
TERMINALS_ENTITY_ID_FIELD = "terminal_id"

<div class="alert alert-block alert-info">
<b>NOTE: Creating Terminal entity can take about 5 to 7 minutes</b>
</div>

In [ ]:
terminal_entity_type.ingest_from_bq(
    feature_ids=TERMINALS_FEATURES_IDS,
    feature_time=FEATURE_TIME,
    bq_source_uri=TERMINALS_BQ_SOURCE_URI,
    entity_id_field=TERMINALS_ENTITY_ID_FIELD,
    disable_online_serving=False,
    worker_count=10,
    sync=True,
)

#### Monitor the ingestion jobs in the console.

The ingestion jobs you just created run asynchronously and they should take several minutes to complete. Please monitoring them in the [console](https://console.cloud.google.com/vertex-ai/ingestion-jobs).


### Search for feature values 
In this section, you'll run a search query on your feature store to validate that some data was ingested, as expected.

In [ ]:
customer_aggregated_features = customer_entity_type.read(
    entity_ids=["5830444124423549", "5469689693941771", "1361459972478769"],
    feature_ids=CUSTOMERS_FEATURES_IDS,
)

In [ ]:
customer_aggregated_features

### END

Now you can go to the next notebook `03_experimentation.ipynb` within vertex_ai directory

## Clean up

In [ ]:
# ff_feature_store.delete(sync=True, force=True)